# Lesson 8 - Creating an Agentic multi-agent system using A2A with BeeAI Framework

In this final code lesson, you will create a comprehensive "Healthcare Concierge" system. You will use [**IBM BeeAI Framework**](https://framework.beeai.dev/) to orchestrate all three agents you have built so far (Policy, Research, and Provider). The BeeAI `RequirementAgent` will act as a router, deciding which A2A agent to hand off to based on the user's complex query. You will use **LiteLLM** via the LangChain adapter to power the orchestrator with Google Gemini.

```mermaid
graph LR
    %% User / Client Layer
    User([User / A2A Client])
    
    %% Main Orchestrator Layer (Lesson 8)
    subgraph OrchestratorLayer [Router/Requirement Agent]
        Concierge["<b>Healthcare Concierge Agent</b><br/>(BeeAI Framework)<br/><code>Port: 9996</code>"]
    end

    subgraph SubAgents [A2A Agent Servers]
        direction TB

        PolicyAgent["<b>Policy Agent</b><br/>(Gemini with A2A SDK)<br/><code>Port: 9999</code>"]
        ResearchAgent["<b>Research Agent</b><br/>(Google ADK)<br/><code>Port: 9998</code>"]

        ProviderAgent["<b>Provider Agent</b><br/>(LangGraph + LangChain)<br/><code>Port: 9997</code>"]
    end

    %% Data & Tools Layer
    subgraph DataLayer [Data Sources & Tools]
        PDF["Policy PDF"]
        Google[Google Search Tool]
        MCPServer["FastMCP Server<br/>(<code>doctors.json</code>)"]
    end
    
    Label_UA["Sends Query - A2A"]
    Label_CP["A2A"]
    Label_CR["A2A"]
    Label_CProv["A2A"]
    Label_MCP["MCP (stdio)"]

    %% -- CONNECTIONS --
    
    User --- Label_UA --> Concierge

    Concierge --- Label_CP --> PolicyAgent
    Concierge --- Label_CR --> ResearchAgent
    Concierge --- Label_CProv --> ProviderAgent
    
    PolicyAgent -- "Reads" --> PDF
    ResearchAgent -- "Calls" --> Google
    
    ProviderAgent --- Label_MCP --> MCPServer

    classDef orchestrator fill:#f9f,stroke:#333,stroke-width:2px;
    classDef agent fill:#e1f5fe,stroke:#0277bd,stroke-width:2px;
    classDef tool fill:#fff3e0,stroke:#ef6c00,stroke-width:1px,stroke-dasharray: 5 5;
    
    classDef protocolLabel fill:#ffffff,stroke:none,color:#000;
    
    class Concierge orchestrator;
    class PolicyAgent,ResearchAgent,ProviderAgent agent;
    class PDF,Google,MCPServer tool;
    
    class Label_UA,Label_CP,Label_CR,Label_CProv,Label_MCP protocolLabel;
```

In [4]:
import asyncio
import os
from typing import Any

from IPython.display import Markdown, display

from helpers import setup_env

setup_env()

## 8.1. Start All Agent Servers

Ensure all three terminals are running their respective agents:
1.  **Policy Agent** (Lesson 2) - `uv run a2a_policy_agent.py`
2.  **Research Agent** (Lesson 4) - `uv run a2a_research_agent.py`
3.  **Provider Agent** (Lesson 6) - `uv run a2a_provider_agent.py`

Open the terminals as instructed below.

<div style="background-color:#e8f0fe; padding:15px; border-left:5px solid #4285f4; border-radius:4px">
    <b>Terminal Access:</b> Please open three new terminal windows in your Jupyter environment to run the servers.
    <br>You can typically do this by selecting <i>File -> New -> Terminal</i> from the menu.
</div>

## 8.2. Define BeeAI Components

Here you will:
1.  Import BeeAI framework components, including `RequirementAgent` and `HandoffTool`. 
2.  Define `A2AAgent` instances for each of your running servers.
3.  Use `check_agent_exists()` to fetch the metadata (AgentCard) from each server.

In [2]:
from typing import Any

from beeai_framework.adapters.a2a.agents import A2AAgent
from beeai_framework.adapters.gemini import GeminiChatModel

# If using Vertex AI
from beeai_framework.adapters.vertexai import VertexAIChatModel  # noqa: F401
from beeai_framework.agents.requirement import RequirementAgent
from beeai_framework.agents.requirement.requirements.conditional import (
    ConditionalRequirement,
)
from beeai_framework.memory import UnconstrainedMemory
from beeai_framework.middleware.trajectory import EventMeta, GlobalTrajectoryMiddleware
from beeai_framework.tools import Tool
from beeai_framework.tools.handoff import HandoffTool
from beeai_framework.tools.think import ThinkTool


class ConciseGlobalTrajectoryMiddleware(GlobalTrajectoryMiddleware):
    def _format_prefix(self, meta: EventMeta) -> str:
        prefix = super()._format_prefix(meta)
        return prefix.rstrip(": ")

    def _format_payload(self, value: Any) -> str:
        return ""


# Log only tool calls
GlobalTrajectoryMiddleware(target=[Tool])

In [5]:
host = os.getenv("AGENT_HOST")
policy_agent_port = os.getenv("POLICY_AGENT_PORT")
research_agent_port = os.getenv("RESEARCH_AGENT_PORT")
provider_agent_port = os.getenv("PROVIDER_AGENT_PORT")
healthcare_agent_port = int(os.getenv("HEALTHCARE_AGENT_PORT"))

In [6]:
policy_agent = A2AAgent(
    url=f"http://{host}:{policy_agent_port}", memory=UnconstrainedMemory()
)
# Run `check_agent_exists()` to fetch and populate AgentCard
asyncio.run(policy_agent.check_agent_exists())
print("\tℹ️", f"{policy_agent.name} initialized")

	ℹ️ InsurancePolicyCoverageAgent initialized


In [7]:
research_agent = A2AAgent(
    url=f"http://{host}:{research_agent_port}", memory=UnconstrainedMemory()
)
asyncio.run(research_agent.check_agent_exists())
print("\tℹ️", f"{research_agent.name} initialized")

	ℹ️ HealthResearchAgent initialized


In [8]:
provider_agent = A2AAgent(
    url=f"http://{host}:{provider_agent_port}", memory=UnconstrainedMemory()
)
asyncio.run(provider_agent.check_agent_exists())
print("\tℹ️", f"{provider_agent.name} initialized")

	ℹ️ HealthcareProviderAgent initialized


## 8.3. Configure the Orchestrator (Healthcare Concierge)

You will now configure the `RequirementAgent`. This agent uses a `GeminiChatModel` (wrapping `ChatLiteLLM` with Gemini) and is equipped with `HandoffTool`s connected to your A2A agents. The instructions explicitly guide the LLM on how to use each specific agent (Research for conditions, Policy for insurance, Provider for doctors) to answer multi-part questions.

In [10]:
healthcare_agent = RequirementAgent(
    name="Healthcare Agent",
    description="A personal concierge for Healthcare Information, customized to your policy.",
    llm=GeminiChatModel(
        "gemini-3-flash-preview",
        allow_parallel_tool_calls=True,
    ),
    # If using Vertex AI
    # llm = VertexAIChatModel(
    #    model_id="gemini-3-flash-preview",
    #    project= os.environ.get("GOOGLE_CLOUD_PROJECT"),
    #    location="global",
    #    allow_parallel_tool_calls=True,
    # ),
    tools=[
        thinktool := ThinkTool(),
        policy_tool := HandoffTool(
            target=policy_agent,
            name=policy_agent.name,
            description=policy_agent.agent_card.description,
        ),
        research_tool := HandoffTool(
            target=research_agent,
            name=research_agent.name,
            description=research_agent.agent_card.description,
        ),
        provider_tool := HandoffTool(
            target=provider_agent,
            name=provider_agent.name,
            description=provider_agent.agent_card.description,
        ),
    ],
    requirements=[
        ConditionalRequirement(
            thinktool,
            force_at_step=1,
            force_after=Tool,
            consecutive_allowed=False,
        ),
        ConditionalRequirement(
            policy_tool,
            consecutive_allowed=False,
            max_invocations=1,
        ),
        ConditionalRequirement(
            research_tool,
            consecutive_allowed=False,
            max_invocations=1,
        ),
        ConditionalRequirement(
            provider_tool,
            consecutive_allowed=False,
            max_invocations=1,
        ),
    ],
    role="Healthcare Concierge",
    instructions=(
        f"""You are a concierge for healthcare services. Your task is to handoff to one or more agents to answer questions and provide a detailed summary of their answers. Be sure that all of their questions are answered before responding.

        IMPORTANT: When returning answers about providers, only output providers provided by `{provider_agent.name}` and only provide insurance information based on the results from `{policy_agent.name}`.

        In your output, put which agent gave you the information!"""
    ),
)

print("\tℹ️", f"{healthcare_agent.meta.name} initialized")

	ℹ️ Healthcare Agent initialized


## 8.4. Run the Full Workflow

Test the system with a complex query that requires information from all three sub-agents.


In [11]:
response = await healthcare_agent.run(
    "I'm based in Austin, TX. How do I get mental health therapy, who are providers near me and what does my insurance cover?"
).middleware(ConciseGlobalTrajectoryMiddleware())
display(Markdown(response.last_message.text))

🤖 RequirementAgent[Healthcare Agent][start]
--> 🔎 ConditionalRequirement[ConditionThink][start]
<-- 🔎 ConditionalRequirement[ConditionThink][success]
--> 🔎 ConditionalRequirement[ConditionInsurancepolicycoverageagent][start]
<-- 🔎 ConditionalRequirement[ConditionInsurancepolicycoverageagent][success]
--> 🔎 ConditionalRequirement[ConditionHealthresearchagent][start]
<-- 🔎 ConditionalRequirement[ConditionHealthresearchagent][success]
--> 🔎 ConditionalRequirement[ConditionHealthcareprovideragent][start]
<-- 🔎 ConditionalRequirement[ConditionHealthcareprovideragent][success]
--> 💬 GeminiChatModel[GeminiChatModel][start]
<-- 💬 GeminiChatModel[GeminiChatModel][success]
--> 🛠️ ThinkTool[think][start]
--> 🛠️ ThinkTool[think][start]
--> 🛠️ ThinkTool[think][start]
--> 🛠️ ThinkTool[think][start]
--> 🛠️ ThinkTool[think][start]
<-- 🛠️ ThinkTool[think][success]
<-- 🛠️ ThinkTool[think][success]
<-- 🛠️ ThinkTool[think][success]
<-- 🛠️ ThinkTool[think][success]
<-- 🛠️ ThinkTool[think][success]
--> 🔎 Co

ChatModelError: Chat Model error

## 8.5. Write the Agent Code to a File

The Concierge agent code is provided in `a2a_healthcare_agent.py`. It includes the A2A Server registration to run it as an A2A Agent.

In [ ]:
from IPython.display import Code, display

display(Code("a2a_healthcare_agent.py"))

## 8.6. Serve the Concierge Agent

Finally, you can register this high-level "Concierge" agent itself as an A2A server. This demonstrates the recursive power of A2A: an agent composed of other A2A agents can itself be exposed as an A2A agent.

Now to activate your configured A2A agent, you would need to run your agent server. You can run the agent server using `uv`:

- Open a terminal as instructed below
- Type `uv run a2a_healthcare_agent.py` to run the server and activate your A2A agent.


<div style="background-color:#e8f0fe; padding:15px; border-left:5px solid #4285f4; border-radius:4px">
    <b>Terminal Access:</b> Please open a new terminal window in your Jupyter environment to run the server.
    <br>You can typically do this by selecting <i>File -> New -> Terminal</i> from the menu.
</div>

## 8.7. Run the Client

In [12]:
agent = A2AAgent(
    url=f"http://{host}:{healthcare_agent_port}", memory=UnconstrainedMemory()
)
response = await agent.run(
    "I'm based in Austin, TX. How do I get mental health therapy near me and what does my insurance cover?"
).middleware(ConciseGlobalTrajectoryMiddleware())
display(Markdown(response.last_message.text))

🤖 A2AAgent[agent_9996][start]
2026-03-01 16:46:53 | WARNING  | beeai_framework.adapters.a2a.agents.agent:run:230 - Task status (TaskState.failed) is not complete.
🤖 A2AAgent[Healthcare Agent][success]


Chat Model error

## 8.8. Resources

- [BeeAI Framework](https://framework.beeai.dev/introduction/welcome)
- [Requirement Agent](https://framework.beeai.dev/modules/agents/requirement-agent)
- [BeeAI Framework GitHub](https://github.com/i-am-bee/beeai-framework)
- [LiteLLM](https://docs.litellm.ai/)